In [523]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [525]:
# https://www.statsmodels.org/stable/index.html
import statsmodels.api as sm

In [527]:
import warnings
warnings.filterwarnings("ignore")

In [529]:
from functools import partial

In [531]:
from dotenv import load_dotenv

from pathlib import Path

env_path = Path("../../.env-live")

if env_path.exists():
    print('envs Loaded')
    load_dotenv(dotenv_path=env_path)


envs Loaded


In [644]:
# 在修改.env-live文件后，重新加载环境变量
from dotenv import load_dotenv
load_dotenv(dotenv_path=env_path, override=True)  # 使用override确保覆盖现有值

# 再次验证环境变量
print("\n修正后的环境变量:")
print("S3_REGION:", os.getenv("JRJ_MODEL_REGISTRY_S3_REGION"))
print("S3_ENDPOINT:", os.getenv("JRJ_MODEL_REGISTRY_S3_ENDPOINT"))
print("S3_BUCKET:", os.getenv("JRJ_MODEL_REGISTRY_S3_BUCKET_NAME"))


修正后的环境变量:
S3_REGION: ca-central-1
S3_ENDPOINT: s3.ca-central-1.wasabisys.com/273-g1
S3_BUCKET: 273-g1


In [646]:
# from jrjModelRegistry import handleDashboard, jrjRouterModelRegistry
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel

In [648]:
from pymongo import MongoClient
import os

mongo_url = os.environ.get("JRJ_MONGODB_MODEL_REGISTRY")
client = MongoClient(mongo_url)

try:
    client.admin.command("ping")
    print("✅ MongoDB connected successfully.")
except Exception as e:
    print("❌ MongoDB connection failed:", e)


✅ MongoDB connected successfully.


In [650]:
# 在 load_dotenv 后添加检查代码
import os
print("验证环境变量:")
print("S3_REGION:", os.getenv("JRJ_MODEL_REGISTRY_S3_REGION"))
print("S3_ENDPOINT:", os.getenv("JRJ_MODEL_REGISTRY_S3_ENDPOINT"))

验证环境变量:
S3_REGION: ca-central-1
S3_ENDPOINT: s3.ca-central-1.wasabisys.com/273-g1


In [652]:
# 临时创建S3客户端测试连接 - 修正版
s3_test = boto3.client(
    's3',
    endpoint_url='https://s3.ca-central-1.wasabisys.com',  # 使用修正后的端点
    region_name=os.getenv("JRJ_MODEL_REGISTRY_S3_REGION"),
    aws_access_key_id=os.getenv("JRJ_MODEL_REGISTRY_S3_KEY_ID"),
    aws_secret_access_key=os.getenv("JRJ_MODEL_REGISTRY_S3_KEY_SECRET"),
    config=Config(signature_version='s3v4')
)

try:
    # 尝试列出bucket内容 - 指定特定bucket
    response = s3_test.list_objects_v2(Bucket='273-g1')
    print("S3连接测试成功！Bucket内容:")
    if 'Contents' in response:
        for obj in response['Contents']:
            print(f"- {obj['Key']}")
    else:
        print("Bucket为空")
except Exception as e:
    print(f"S3连接测试失败: {str(e)}")

S3连接测试成功！Bucket内容:
- 273-g1/Group1_regression_model1__1.0.0.pkl.zip
- 273-g1/salaryRegModel1Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_1_annArborRegModel1Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_1_annArborRegModel2Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_1_annArborRegModel3Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_1_annArborRegModel4Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_1_annArborRegModelBEST__1.0.1.pkl.zip
- 273-g1/teng_zhang_annArborRegModel1Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_annArborRegModel2Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_annArborRegModel3Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_annArborRegModel4Fit_RF__1.0.1.pkl.zip
- 273-g1/teng_zhang_annArborRegModel4Fit_RT1__1.0.1.pkl.zip
- 273-g1/teng_zhang_annArborRegModel4Fit_RT2__1.0.1.pkl.zip
- 273-g1/teng_zhang_annArborRegModel4Fit__1.0.1.pkl.zip
- 273-g1/teng_zhang_annArborRegModelBEST__1.0.1.pkl.zip
- 273-g1/teng_zhang_dTModel1__1.0.1.pkl.zip
- 273-g1/teng_zhang_dTModelBestAccuracy__1.0.1.pkl.zip
- 273-g1/teng_zhang_dTModelBestF1__1.0.1.

In [654]:
def generalRegressionPredictor(self,transformedData):
    return self.predict(transformedData)
# mainpredictor function

In [656]:
sallaryMisDf = pd.read_excel("https://www.dropbox.com/scl/fi/zt2vtwhpz8ndblsxqdqx1/Salary_MIS.xlsx?rlkey=2uk6m7m9w90isv6zsynhhhpyv&st=gxumjns5&dl=1")

In [657]:
sallaryMisDf

,Salary,GPA,MIS,Statistics
0,72,3.53,1,0
1,66,2.86,1,0
2,72,3.69,0,0
3,63,3.24,0,0
4,65,3.21,0,0
...,...,...,...,...
115,66,3.27,0,0
116,63,2.86,1,0
117,78,3.04,1,1
118,64,2.99,0,0


In [599]:
sallaryMisDf.shape

(120, 4)

In [600]:
sallaryMisDf.describe()

,Salary,GPA,MIS,Statistics
count,120.000000,120.000000,120.000000,120.000000
mean,69.875000,3.242750,0.316667,0.341667
std,6.594577,0.493834,0.467127,0.476257
min,53.000000,2.410000,0.000000,0.000000
25%,65.750000,2.805000,0.000000,0.000000
50%,70.000000,3.280000,0.000000,0.000000
75%,73.250000,3.692500,1.000000,1.000000
max,88.000000,3.980000,1.000000,1.000000


In [601]:
# creat transformer function
def salaryRegModel1Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    if isinstance(dataForTransfer, pd.DataFrame):
        df=dataForTransfer.copy()
    else:
        df=pd.DataFrame(dataForTransfer)
    dfTransformer = sm.add_constant(df[['GPA','MIS','Statistics']], has_constant="add")
    return dfTransformer
# repeatitive for most of the models

In [602]:
sm.add_constant(sallaryMisDf[['GPA','MIS','Statistics']])

,const,GPA,MIS,Statistics
0,1.0,3.53,1,0
1,1.0,2.86,1,0
2,1.0,3.69,0,0
3,1.0,3.24,0,0
4,1.0,3.21,0,0
...,...,...,...,...
115,1.0,3.27,0,0
116,1.0,2.86,1,0
117,1.0,3.04,1,1
118,1.0,2.99,0,0


In [603]:
sallaryMisDf.shape

(120, 4)

In [604]:
sm.add_constant(sallaryMisDf[["GPA", "MIS", "Statistics"]])

,const,GPA,MIS,Statistics
0,1.0,3.53,1,0
1,1.0,2.86,1,0
2,1.0,3.69,0,0
3,1.0,3.24,0,0
4,1.0,3.21,0,0
...,...,...,...,...
115,1.0,3.27,0,0
116,1.0,2.86,1,0
117,1.0,3.04,1,1
118,1.0,2.99,0,0


In [605]:
# salaryRegModel1 = sm.OLS(
#   sallaryMisDf["Salary"], #dependent 
#   sm.add_constant(sallaryMisDf[["GPA", "MIS", "Statistics"]])
# )

In [660]:
# calling my fuynction
salaryRegModel1 = sm.OLS(
  sallaryMisDf["Salary"], #dependent 
  salaryRegModel1Transformer(sallaryMisDf)
)

In [662]:
salaryRegModel1Fit = salaryRegModel1.fit()

In [664]:
print(salaryRegModel1Fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.790
Method:                 Least Squares   F-statistic:                     150.3
Date:                Tue, 08 Jul 2025   Prob (F-statistic):           8.35e-40
Time:                        19:34:57   Log-Likelihood:                -300.92
No. Observations:                 120   AIC:                             609.8
Df Residuals:                     116   BIC:                             621.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         44.0072      1.860     23.662      0.0

In [609]:
sallaryMisDf

,Salary,GPA,MIS,Statistics
0,72,3.53,1,0
1,66,2.86,1,0
2,72,3.69,0,0
3,63,3.24,0,0
4,65,3.21,0,0
...,...,...,...,...
115,66,3.27,0,0
116,63,2.86,1,0
117,78,3.04,1,1
118,64,2.99,0,0


In [666]:
salaryRegModel1SampleData = {
    "GPA": [3.53],
    "MIS": [1],
    "Statistics": [0]
}

In [668]:
salaryRegModel1Fit.transformer = salaryRegModel1Transformer

In [670]:
test1 = salaryRegModel1Fit.transformer(salaryRegModel1SampleData)
test1

,const,GPA,MIS,Statistics
0,1.0,3.53,1,0


In [672]:
salaryRegModel1Fit.mainPredictor = partial(generalRegressionPredictor,salaryRegModel1Fit)

In [676]:
salaryRegModel1Fit.mainPredictor(test1)

0    73.992468
dtype: float64

In [678]:
salaryRegModel1FitMetadata = {
    "modelName":f"zihan_su__salaryRegModel1Fit",
    "version":"1.0.1",
    "params": salaryRegModel1Fit.params.to_dict(),
    "score": float(salaryRegModel1Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel1Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel1Fit.rsquared),
        "adj_r_squared": float(salaryRegModel1Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel1SampleData
    }
}
salaryRegModel1FitMetadata

{'modelName': 'zihan_su__salaryRegModel1Fit',
 'version': '1.0.1',
 'params': {'const': 44.007246340574866,
  'GPA': 6.622706150356169,
  'MIS': 6.607068893822079,
  'Statistics': 6.7309311433580135},
 'score': 0.7953924097617404,
 'modelLibraray': 'sm.OLS',
 'libraryMetadata': {'pvalues': {'const': 3.2704010980896686e-46,
   'GPA': 3.0777930774061555e-21,
   'MIS': 6.058766938187595e-20,
   'Statistics': 1.237978135734885e-20},
  'r_squared': 0.7953924097617404,
  'adj_r_squared': 0.7901008341521303},
 'sampleData': {'dataForTransfer': {'GPA': [3.53],
   'MIS': [1],
   'Statistics': [0]}}}

In [684]:
# 临时覆盖环境变量
os.environ["JRJ_MODEL_REGISTRY_S3_ENDPOINT"] = "s3.ca-central-1.wasabisys.com"

# 重新创建S3客户端（如果需要）
from jrjModelRegistry.jrjModelRegistry import registerAJrjModel
registerAJrjModel(salaryRegModel1Fit, salaryRegModel1FitMetadata)

❌ Upload failed via PUT: 403 <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>SignatureDoesNotMatch</Code><Message>The request signature we calculated does not match the signature you provided. Check your key and signing method.</Message><AWSAccessKeyId>Q57X4P0GR5L8FI6580K7</AWSAccessKeyId><StringToSign>PUT&#xA;&#xA;&#xA;1751975857&#xA;/273-g1/273-g1/zihan_su__salaryRegModel1Fit__1.0.1.pkl.zip</StringToSign><SignatureProvided>8/sDPBQWKpQlZb1TmziYunMetRc=</SignatureProvided><StringToSignBytes>50 55 54 a a a 31 37 35 31 39 37 35 38 35 37 a 2f 32 37 33 2d 67 31 2f 32 37 33 2d 67 31 2f 7a 69 68 61 6e 5f 73 75 5f 5f 73 61 6c 61 72 79 52 65 67 4d 6f 64 65 6c 31 46 69 74 5f 5f 31 2e 30 2e 31 2e 70 6b 6c 2e 7a 69 70</StringToSignBytes><RequestId>F064F7994C144553:A</RequestId><HostId>l9vypW1DemBzJIX7VJXpeEjmSq1hTGcriIiOIbOfhr/qdUaC0kUsH7RqnKZZ5Edd6tviinNIgkLl</HostId><CMReferenceId>MTc1MTk3NTI1ODg0MCAzOC4xNDMuMTQ2LjEwMiBDb25JRDoxODYyMDE1MTM2L0VuZ2luZUNvbklEOjE3ODY4MjcyL0NvcmU6OTA=</CMReferen

In [680]:
registerAJrjModel(salaryRegModel1Fit, salaryRegModel1FitMetadata)

❌ Upload failed via PUT: 403 <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>SignatureDoesNotMatch</Code><Message>The request signature we calculated does not match the signature you provided. Check your key and signing method.</Message><AWSAccessKeyId>Q57X4P0GR5L8FI6580K7</AWSAccessKeyId><StringToSign>PUT&#xA;&#xA;&#xA;1751975263&#xA;/273-g1/273-g1/zihan_su__salaryRegModel1Fit__1.0.1.pkl.zip</StringToSign><SignatureProvided>JEVS/+89wLJa0+Y5HI7eUPWLghk=</SignatureProvided><StringToSignBytes>50 55 54 a a a 31 37 35 31 39 37 35 32 36 33 a 2f 32 37 33 2d 67 31 2f 32 37 33 2d 67 31 2f 7a 69 68 61 6e 5f 73 75 5f 5f 73 61 6c 61 72 79 52 65 67 4d 6f 64 65 6c 31 46 69 74 5f 5f 31 2e 30 2e 31 2e 70 6b 6c 2e 7a 69 70</StringToSignBytes><RequestId>1CDADDF9E0A6B06A:B</RequestId><HostId>IHOZA7eeALczEIGD43yOoKIYgErG2hL0X0qlNjhs6lkWQYjC7uFAzAHg5xhdp60gCd3vSF1+OiPX</HostId><CMReferenceId>MTc1MTk3NDY2NDQ5MyAzOC4xNDMuMTQ2LjEwMyBDb25JRDoxODYxOTA3NDYxL0VuZ2luZUNvbklEOjE3ODc3ODIzL0NvcmU6NjU=</CMReferen

In [ ]:
sallaryMisDf

In [ ]:
sallaryMisDf["misXStatistics"] = sallaryMisDf["MIS"] * sallaryMisDf["Statistics"]
sallaryMisDf['misXStatistics1'] = sallaryMisDf.apply(lambda row: row['MIS'] * row['Statistics'], axis=1)

sallaryMisDf

In [ ]:
salaryRegModel2 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXStatistics1"
      ]]
  )
)

In [ ]:
salaryRegModel2Fit = salaryRegModel2.fit()

In [ ]:
print(salaryRegModel2Fit.summary())

In [ ]:
salaryRegModel2SampleData = salaryRegModel1SampleData

In [ ]:
def salaryRegModel2Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer['misXStatistics1'] = dfTransformer.apply(lambda row: row['MIS'] * row['Statistics'], axis=1)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics', 'misXStatistics1']])
    return dfTransformer

In [ ]:
salaryRegModel2Fit.transformer = salaryRegModel2Transformer

In [ ]:
salaryRegModel2Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel2Fit)

In [ ]:
test2 = salaryRegModel2Fit.transformer(salaryRegModel2SampleData)
test2

In [ ]:
salaryRegModel2Fit.mainPredictor(test2)

In [ ]:
salaryRegModel2FitMetadata = {
    "modelName":f"salaryRegModel2Fit",
    "version":"1.0.1",
    "params": salaryRegModel2Fit.params.to_dict(),
    "score": float(salaryRegModel2Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel2Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel2Fit.rsquared),
        "adj_r_squared": float(salaryRegModel2Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel2SampleData
    }
}
salaryRegModel2FitMetadata

In [ ]:
registerAJrjModel(salaryRegModel2Fit, salaryRegModel2FitMetadata)

In [ ]:
sallaryMisDf['misXGpa'] = sallaryMisDf.apply(lambda row: row['MIS'] * row['GPA'], axis=1)

sallaryMisDf


In [ ]:
salaryRegModel3 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel3Fit = salaryRegModel3.fit()

In [ ]:
print(salaryRegModel3Fit.summary())

In [ ]:
salaryRegModel3Fit = salaryRegModel3.fit()

In [ ]:
print(salaryRegModel3Fit.summary())

In [ ]:
sallaryMisDf['statisticsXGpa'] = sallaryMisDf.apply(lambda row: row['Statistics'] * row['GPA'], axis=1)

sallaryMisDf

In [ ]:
salaryRegModel4 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel4Fit = salaryRegModel4.fit()

In [ ]:
print(salaryRegModel4Fit.summary())

In [ ]:
salaryRegModel5 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          "MIS",
          "Statistics",
          "misXStatistics1",
          "misXGpa",
          "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel5Fit = salaryRegModel5.fit()

In [ ]:
print(salaryRegModel5Fit.summary())

In [ ]:
salaryRegModel6 = sm.OLS(
  sallaryMisDf["Salary"],
  sm.add_constant(
      sallaryMisDf[[
          "GPA",
          # "MIS",
          "Statistics",
          "misXStatistics1",
          # "misXGpa",
          # "statisticsXGpa"
      ]]
  )
)

In [ ]:
salaryRegModel6Fit = salaryRegModel6.fit()

In [ ]:
print(salaryRegModel6Fit.summary())

## AFTER AREFULLY LOOKUP WE RELIZED  salaryRegModel4Fit SO LETS KEEP IT AS THE BEST

In [ ]:
def salaryRegModel4Transformer(dataForTransfer = None):
    import pandas as pd
    import statsmodels.api as sm
    dfTransformer = pd.DataFrame(dataForTransfer)
    dfTransformer['statisticsXGpa'] = dfTransformer.apply(lambda row: row['Statistics'] * row['GPA'], axis=1)
    dfTransformer = sm.add_constant(dfTransformer[['GPA', 'MIS', 'Statistics', 'statisticsXGpa']])
    return dfTransformer

In [ ]:
salaryRegModel4Fit.transformer = salaryRegModel4Transformer

In [ ]:
salaryRegModel4Fit.mainPredictor = partial(salaryRegModel1MainPredictor, salaryRegModel4Fit)

In [ ]:
test4 = salaryRegModel4Fit.transformer(salaryRegModel2SampleData)
test4

In [ ]:
salaryRegModel4Fit.mainPredictor(test4)

In [ ]:
salaryRegModel4FitMetadata = {
    "modelName":f"salaryRegModel4Fit",
    "version":"1.0.1",
    "params": salaryRegModel4Fit.params.to_dict(),
    "score": float(salaryRegModel4Fit.rsquared),
    "modelLibraray": 'sm.OLS',
    "libraryMetadata": {
        "pvalues": salaryRegModel4Fit.pvalues.to_dict(),
        "r_squared": float(salaryRegModel4Fit.rsquared),
        "adj_r_squared": float(salaryRegModel4Fit.rsquared_adj)
    },

    "sampleData": {
        "dataForTransfer": salaryRegModel2SampleData
    }
}
salaryRegModel4FitMetadata

In [ ]:
registerAJrjModel(salaryRegModel4Fit, salaryRegModel4FitMetadata)

In [ ]:
sallaryMisDf
